In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Importing station dataframe
station_df = pd.read_csv('station_info.csv')
station_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 4 columns):
Abbreviation    46 non-null object
Description     46 non-null object
Location        46 non-null object
Name            46 non-null object
dtypes: object(4)
memory usage: 1.6+ KB


In [3]:
station_df.head()

Abbreviation  ...                                 Name
0         12TH  ...  12th St. Oakland City Center (12TH)
1         16TH  ...              16th St. Mission (16TH)
2         19TH  ...              19th St. Oakland (19TH)
3         24TH  ...              24th St. Mission (24TH)
4         ASHB  ...                         Ashby (ASHB)

[5 rows x 4 columns]

In [4]:
#Importing BART 2016 dataframe
df_2016 = pd.read_csv('date-hour-soo-dest-2016.csv')
df_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9971582 entries, 0 to 9971581
Data columns (total 4 columns):
Origin         object
Destination    object
Throughput     int64
DateTime       object
dtypes: int64(1), object(3)
memory usage: 304.3+ MB


In [5]:
df_2016.head()

Origin Destination  Throughput             DateTime
0   12TH        12TH           1  2016-01-01 00:00:00
1   12TH        16TH           1  2016-01-01 00:00:00
2   12TH        24TH           4  2016-01-01 00:00:00
3   12TH        ASHB           4  2016-01-01 00:00:00
4   12TH        BALB           2  2016-01-01 00:00:00

In [6]:
#Importing BART 2017 dataframe
df_2017 = pd.read_csv('date-hour-soo-dest-2017.csv')
df_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3313625 entries, 0 to 3313624
Data columns (total 4 columns):
Origin         object
Destination    object
Throughput     int64
DateTime       object
dtypes: int64(1), object(3)
memory usage: 101.1+ MB


In [7]:
df_2017.head()

Origin Destination  Throughput             DateTime
0   12TH        19TH           1  2017-01-01 00:00:00
1   12TH        24TH           2  2017-01-01 00:00:00
2   12TH        BAYF           1  2017-01-01 00:00:00
3   12TH        CIVC           5  2017-01-01 00:00:00
4   12TH        COLS           2  2017-01-01 00:00:00

In [8]:
#Append BART 2016 and 2017 dataframe
df = df_2016.append(df_2017, ignore_index = True)
df.head()

Origin Destination  Throughput             DateTime
0   12TH        12TH           1  2016-01-01 00:00:00
1   12TH        16TH           1  2016-01-01 00:00:00
2   12TH        24TH           4  2016-01-01 00:00:00
3   12TH        ASHB           4  2016-01-01 00:00:00
4   12TH        BALB           2  2016-01-01 00:00:00

In [0]:
#Renaming Abbreviation, Location and Name columns in order to distinguish Destination stations prior to merging station 
#dataframe with the main dataframe
dest_df = station_df[['Abbreviation', 'Location', 'Name']]
dest_df = dest_df.rename(columns = {'Abbreviation': 'Destination', 'Location' : 'Dest_Location', 'Name': 'Dest_Name'})

In [0]:
#Merging Destination station dataframe with main dataframe for Destination
df = df.merge(dest_df, how = 'left', on = 'Destination')

In [0]:
#Renaming station dataframe columns to distinguish Origin station info prior to merging to main dataframe
origin_df = station_df[['Abbreviation', 'Location', 'Name']]
origin_df = origin_df.rename(columns = {'Abbreviation': 'Origin', 'Location': 'Origin_Loc', 'Name': 'Origin_Name'})

In [12]:
#Merging Origin station dataframe with main dataframe
df = df.merge(origin_df, how = 'left', on = 'Origin')
df.isnull().sum()

Origin               0
Destination          0
Throughput           0
DateTime             0
Dest_Location    16551
Dest_Name        16551
Origin_Loc       15440
Origin_Name      15440
dtype: int64

In [13]:
#Viewing data that is null for Origin location, which all happens to be WSPR
#df[df['Origin_Loc'].isnull()]
Origin_null = df[df['Origin_Loc'].isnull()]
Origin_null['Origin'].unique()

array(['WSPR'], dtype=object)

In [14]:
#Double checking if WSPR was originally in station dataframe
Origin_null['Origin'].isin(station_df['Abbreviation']).value_counts()

False    15440
Name: Origin, dtype: int64

In [0]:
#Dropping all null cells from Origin/Destination stations that do not exist in station dataframe
df.dropna(axis = 0, inplace = True)

In [0]:
#Turning longitude and latitude into tuples
df['Dest_Location'] = df['Dest_Location'].apply(lambda x: x.split(',')[:2])
df['Origin_Loc'] = df['Origin_Loc'].apply(lambda x: x.split(',')[:2])

In [17]:
#Viewing the number of unique values in column Throughput and the data that has the highest number of throughput
print('the number of unique values is: ', df['Throughput'].nunique())
df[df['Throughput'] == df['Throughput'].max()]

the number of unique values is:  950


Origin Destination  ...                Origin_Loc              Origin_Name
10504519   24TH        CIVC  ...  [-122.418143, 37.752470]  24th St. Mission (24TH)

[1 rows x 8 columns]

In [0]:
#Which Origin station is the busiest?
#Started by grouping by the sum of throughputs by station for Origins and Destinations
origin = df.groupby(['Origin'])['Throughput'].sum().sort_values(ascending = False)
destination = df.groupby(['Destination'])['Throughput'].sum().sort_values(ascending = False)

In [0]:
#Turning the groupby's into dataframes
origin_station = pd.DataFrame(data = origin)
destination_station = pd.DataFrame(data = destination)

In [20]:
#Built a dataframe that merged both groupby's dataframes
busy_station = origin_station.merge(destination_station, how = 'left', left_index = True, right_index = True).reset_index()
busy_station.rename(columns = {'Origin': 'Station', 'Throughput_x': 'Origin_Throughput', 'Throughput_y': 'Dest_Throughput'},
                    inplace = True)
busy_station.head()

Station  Origin_Throughput  Dest_Throughput
0    MONT           16092804         16942893
1    EMBR           15733036         18308407
2    POWL           14131735         12449029
3    CIVC           10139171          9386011
4    24TH            5458712          5302403

In [21]:
#Summed throughputs from both Origin and Destination for a total number of throughputs and printed the station with 
#the highest sum of throughputs
busy_station['Sum_Throughput'] = busy_station['Origin_Throughput'] + busy_station['Dest_Throughput']
busy_station[busy_station['Sum_Throughput'] == busy_station['Sum_Throughput'].max()]

Station  Origin_Throughput  Dest_Throughput  Sum_Throughput
1    EMBR           15733036         18308407        34041443

In [22]:
#Least popular BART station
busy_station[busy_station['Sum_Throughput'] == busy_station['Sum_Throughput'].min()]

Station  Origin_Throughput  Dest_Throughput  Sum_Throughput
44    OAKL             698694           620832         1319526

In [0]:
import time
import datetime
df['DateTime'] = df['DateTime'].apply(lambda x: datetime.datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S').timetuple())

In [24]:
#Day of the week that is busiest
df['WeekDay'] = df['DateTime'].apply(lambda x: x[6])
df['WeekDay'].mode()

0    4
dtype: int64

In [25]:
#Number of people taking BART late at night
df['Hour'] = df['DateTime'].apply(lambda x: x[3])

late = df.loc[(df['Hour'] <= 3) | (df['Hour'] == 23)]
print( 'The average number of people taking the BART late at night is: ', late.groupby('Hour')['Throughput'].mean().sum())

The average number of people taking the BART late at night is:  30.096275014418634


In [0]:
df['DateTime'] = df['DateTime'].apply(lambda x: time.mktime(x))

In [0]:
df.drop(['WeekDay', 'Hour'], axis = 1, inplace = True)

In [0]:
df.drop(['Dest_Name', 'Origin_Name'], axis = 1, inplace = True)

In [0]:
df_2 = pd.get_dummies(df[['Origin', 'Destination', 'Throughput', 'DateTime']], drop_first = True)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

y,x = df_2['Throughput'], df_2.drop('Throughput', axis = 1)

In [0]:
GBR = GradientBoostingRegressor(learning_rate = 0.1, random_state = 0, n_estimators=300)
RFR = RandomForestRegressor(n_estimators = 100, max_depth = 3, random_state = 0)
XGB = xgb.XGBRegressor(n_estimators=100, learning_rate=0.01, max_depth = 3, random_state=0)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 10)

In [0]:
algorithms = [RFR]
reg_type = ['RFR']

def regressions():
    RMSE = []
    R2 = []
    y_pred = []
    
    for i in range(len(algorithms)):
        y_pred = (algorithms[i].fit(x_train, y_train).predict(x_test))
        RMSE.append((mean_squared_error(y_test, y_pred))**0.5)
        R2.append(r2_score(y_test, y_pred)) 

    metrics = pd.DataFrame(columns = ['RMSE', 'R2'], index = reg_type)
    metrics['RMSE'] = RMSE
    metrics['R2'] = R2  
    return metrics

In [35]:
regressions()

RMSE        R2
RFR  32.136686  0.085524